In [1]:
f = open("../api_key.txt", "r")
api_key = f.readline()

In [2]:
import requests
import json
from os.path import exists
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from data_manager import grab_daily_data, get_stock_data, preprocess_data, preprocess_data_multiple

In [17]:
def get_compiled_model():
    model = keras.Sequential()
    #model.add(layers.Embedding(input_dim=(365,0), output_dim=(1,)))
    model.add(layers.LSTM(128))
    model.add(layers.Dense(64, activation='relu'))
    model.compile(loss='mean_squared_error',
                  metrics=[tf.keras.metrics.MeanSquaredError()],
                  optimizer='adam')
    return model

In [4]:
def train_model(symbol, model, api_key, start=None, end=None):
    if (start is None and end is not None) or (start is not None and end is None):
        raise ValueError("start and end has to be both non-None or both None")
    x = y = []
    if not isinstance(symbol, str):
        x, y = preprocess_data_multiple(symbol, api_key)
    else:
        x, y, time_series = preprocess_data(symbol, api_key, start=start, end=end)
    model.fit(np.asarray(x), np.asarray(y), epochs=30, batch_size=128)
    return model

In [18]:
model = get_compiled_model()
train_model(["IBM", "META", "DIS"], model, api_key)

Epoch 1/30


ValueError: in user code:

    File "D:\Projects\stock_prediction\env\lib\site-packages\keras\engine\training.py", line 1051, in train_function  *
        return step_function(self, iterator)
    File "D:\Projects\stock_prediction\env\lib\site-packages\keras\engine\training.py", line 1040, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "D:\Projects\stock_prediction\env\lib\site-packages\keras\engine\training.py", line 1030, in run_step  **
        outputs = model.train_step(data)
    File "D:\Projects\stock_prediction\env\lib\site-packages\keras\engine\training.py", line 889, in train_step
        y_pred = self(x, training=True)
    File "D:\Projects\stock_prediction\env\lib\site-packages\keras\utils\traceback_utils.py", line 67, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "D:\Projects\stock_prediction\env\lib\site-packages\keras\engine\input_spec.py", line 214, in assert_input_compatibility
    

In [12]:
x, y = preprocess_data_multiple(['IBM', 'META'], api_key)
len(x[0])

365